# Detecting fruits in images with RetinaNet

#### Introduction
> Object detection a very important problem in computer vision. Here the model is tasked with localizing the objects present in an image, and at the same time, classifying them into different categories. Object detection models can be broadly classified into "single-stage" and "two-stage" detectors. Two-stage detectors are often more accurate but at the cost of being slower. Here in this example, we will implement RetinaNet, a popular single-stage detector, which is accurate and runs fast. RetinaNet uses a feature pyramid network to efficiently detect objects at multiple scales and introduces a new loss, the Focal loss function, to alleviate the problem of the extreme foreground-background class imbalance.

### source: https://keras.io/examples/vision/retinanet/

## Imports

In [1]:
import os
from os.path import exists
import re
import zipfile

import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

## Data

In [2]:
# Public data
#!curl -L "https://public.roboflow.com/ds/7TKfhyTJFU?key=ItOmD7UQNC" > roboflow.zip; unzip roboflow.zip -d "../data"; rm roboflow.zip

# Our data
!curl -L "https://app.roboflow.com/ds/oycQ6EMCsQ?key=guC6RjJNhS" > roboflow.zip; unzip roboflow.zip -d "../data"; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   892  100   892    0     0    971      0 --:--:-- --:--:-- --:--:--   971
100  132M  100  132M    0     0  4283k      0  0:00:31  0:00:31 --:--:-- 4376k    0  3281k      0  0:00:41  0:00:09  0:00:32 3827k
Archive:  roboflow.zip
 extracting: ../data/README.roboflow.txt  
 extracting: ../data/test/100_jpg.rf.5e0f405a59b380962749e5dbd68ca728.jpg  
 extracting: ../data/test/1014_jpg.rf.8d24b2c8c9d909f07674d707d58ff98f.jpg  
 extracting: ../data/test/1062_jpg.rf.7baf30f46300d0614a4398293893cc98.jpg  
 extracting: ../data/test/1064_jpg.rf.0d567323574b924cb3892aa556d9a479.jpg  
 extracting: ../data/test/1067_jpg.rf.60d6562d21c30e2dbf1a3d806afc7d26.jpg  
 extracting: ../data/test/1071_jpg.rf.3056c73c2472ddae1a1d4cee0d8a021f.jpg  
 extracting: ../data/test/1075_jpg.rf.45b221d241fdafb91a76bce6be885604.jpg  
 extracting: ../data/test/10

 extracting: ../data/test/2888_jpg.rf.0f480380328a9a87026eb3777eaea134.jpg  
 extracting: ../data/test/2901_jpg.rf.ce439ead70b628f3703866947f8a79b1.jpg  
 extracting: ../data/test/2902_jpg.rf.6d95d3c423371f31d7bf868eb95ab318.jpg  
 extracting: ../data/test/2908_jpg.rf.bd0911db1ce7d0dce4075b335639ec64.jpg  
 extracting: ../data/test/290_jpg.rf.10ba2d94939b1873646fd59e5b8de1b0.jpg  
 extracting: ../data/test/2910_jpg.rf.2d806efae3172761114035ca2022680b.jpg  
 extracting: ../data/test/2917_jpg.rf.65c7534f7ac73b2cb24455948a233d9f.jpg  
 extracting: ../data/test/2973_jpg.rf.9fefa3ef6ba1799462be57278553dfd7.jpg  
 extracting: ../data/test/2979_jpg.rf.082b03de39fa9fd84ecbba2651114be1.jpg  
 extracting: ../data/test/3016_jpg.rf.1525f6183944dd1c72e0ed81c784e868.jpg  
 extracting: ../data/test/3022_jpg.rf.8acba2cb0c15eeaa78b70973f659963e.jpg  
 extracting: ../data/test/3026_jpg.rf.1bbfddd57341ede5556e835ca6862b18.jpg  
 extracting: ../data/test/3028_jpg.rf.6a825406cf215e07017c23483eb7dc97.jpg  


 extracting: ../data/test/706_jpg.rf.02eec84d0fe2e854988ef6852e0f9552.jpg  
 extracting: ../data/test/709_jpg.rf.13d3ed721cb3ddc496e2de8cd59bff47.jpg  
 extracting: ../data/test/727_jpg.rf.3a4f748463abb6bf3c1ad14dadadabeb.jpg  
 extracting: ../data/test/728_jpg.rf.6c5d6820ecc0138dfbfee9170b1e643d.jpg  
 extracting: ../data/test/731_jpg.rf.1188e1a6fe90f8dcda7e84ea17f03842.jpg  
 extracting: ../data/test/743_jpg.rf.c89701e9024297c2f318f7b625f521d0.jpg  
 extracting: ../data/test/746_jpg.rf.697383dd6c7b9e03409777146bef9492.jpg  
 extracting: ../data/test/748_jpg.rf.f9b9c78c38dc73610145b12b647f5c2b.jpg  
 extracting: ../data/test/755_jpg.rf.a900170b2afb8011b9eed245d869fbc1.jpg  
 extracting: ../data/test/766_jpg.rf.02cd4858127a1463e8939203996ff6b5.jpg  
 extracting: ../data/test/768_jpg.rf.ff816627ec0a137f792272cea2c76cd6.jpg  
 extracting: ../data/test/76_jpg.rf.6432046d7e720555261fe06f9e19149e.jpg  
 extracting: ../data/test/770_jpg.rf.70f7aafd084a18bf6b318438e24f8490.jpg  
 extracting: 

 extracting: ../data/train/1269_jpg.rf.a6e7a6d968b09129bac5b8097727e5b3.jpg  
 extracting: ../data/train/126_jpg.rf.0fc131bb8fad7dee957756ab5067d009.jpg  
 extracting: ../data/train/1270_jpg.rf.d7f4dacd84a53b66905a6c7ab6d1933b.jpg  
 extracting: ../data/train/1273_jpg.rf.3026c133597bfc4e6daea28a6e4f440b.jpg  
 extracting: ../data/train/1274_jpg.rf.69e6ec68a545acc2235fdf5deb97cb13.jpg  
 extracting: ../data/train/1275_jpg.rf.9ae5a4f43ba2e3b444f996069409de16.jpg  
 extracting: ../data/train/1276_jpg.rf.3991a4a19d09f764476575bf94a0667a.jpg  
 extracting: ../data/train/1277_jpg.rf.d66f3a42433a6dbe243253bb10c40dc3.jpg  
 extracting: ../data/train/1278_jpg.rf.e5cf90be8dff311af852c0fc94b06f7b.jpg  
 extracting: ../data/train/1279_jpg.rf.43425338a97459d18e421ca464c92d0a.jpg  
 extracting: ../data/train/127_jpg.rf.79dbe134cd604264f7cb473c36474a55.jpg  
 extracting: ../data/train/1280_jpg.rf.486034bf3dee27979f8d2f1a80357302.jpg  
 extracting: ../data/train/1281_jpg.rf.4b467c2f4deb7c24630759943fa

 extracting: ../data/train/1618_jpg.rf.938503c1f935593472c6bb2866522f2a.jpg  
 extracting: ../data/train/161_jpg.rf.14292b26adbd3f2814eef4d187348381.jpg  
 extracting: ../data/train/1620_jpg.rf.4cb681388a2a50891e04f2aa62fef7fd.jpg  
 extracting: ../data/train/1621_jpg.rf.12724b7e52fb03c0b406ea170110cb32.jpg  
 extracting: ../data/train/1622_jpg.rf.acd54673c93d64b73735e23003d69116.jpg  
 extracting: ../data/train/1624_jpg.rf.9a725d2ab1eed8c500b0508ed1f52cf5.jpg  
 extracting: ../data/train/1625_jpg.rf.ffeb8feab96f1f43d5cc73dbafa897c1.jpg  
 extracting: ../data/train/1627_jpg.rf.8ceb780c2c0b53fb335383cb45573843.jpg  
 extracting: ../data/train/1629_jpg.rf.4b2418dd262cb4aecdafc8bbae030642.jpg  
 extracting: ../data/train/162_jpg.rf.d5c8ee8002758ba313c34266957f8321.jpg  
 extracting: ../data/train/1630_jpg.rf.fdd91d2e261bccaa45fba40be36115c3.jpg  
 extracting: ../data/train/1632_jpg.rf.faa3701b5d05fef346cde7ef2c20e993.jpg  
 extracting: ../data/train/1633_jpg.rf.116641942710e60b04cb6becc96

...

 extracting: ../data/train/591_jpg.rf.bb212f9af45711eb8ad93cf7c11b934c.jpg  
 extracting: ../data/train/592_jpg.rf.c1ff03afe5616b72459f6796784b3aab.jpg  
 extracting: ../data/train/593_jpg.rf.50bb665b131632f79132c625fcf675bb.jpg  
 extracting: ../data/train/594_jpg.rf.6741661b6346f8bcbd5478bb44ba8dff.jpg  
 extracting: ../data/train/595_jpg.rf.1af032b0aaec4f8d19242bf64852d915.jpg  
 extracting: ../data/train/598_jpg.rf.6f3512f887ccd8607ab9b635c31af96e.jpg  
 extracting: ../data/train/599_jpg.rf.45cc6cb5cec1f2d1c30f3100046eee98.jpg  
 extracting: ../data/train/59_jpg.rf.27b883aab6cfa511e2119d80a9ee9f4c.jpg  
 extracting: ../data/train/5_jpg.rf.cc82b7e6673095129fab6edfaf45a20a.jpg  
 extracting: ../data/train/600_jpg.rf.cf8e298d53d6df8f442adc1d362d977b.jpg  
 extracting: ../data/train/601_jpg.rf.2db912322b8c03950c1465234ed0aea7.jpg  
 extracting: ../data/train/602_jpg.rf.27c0fbe9e1d6d7545fbad1b57ce67c92.jpg  
 extracting: ../data/train/604_jpg.rf.16bb93c6f61f58e47c08615bb5afc2fb.jpg  
 e

 extracting: ../data/train/944_jpg.rf.6be230e1face4513dd2561b8d6aef31c.jpg  
 extracting: ../data/train/945_jpg.rf.05970dcf4e78f28bb38de12166396090.jpg  
 extracting: ../data/train/946_jpg.rf.e3d4b492538c40f4805ed3d88a0302a9.jpg  
 extracting: ../data/train/947_jpg.rf.110f0fe7ce0755b190f07cda89ceaaa7.jpg  
 extracting: ../data/train/94_jpg.rf.f1a60605f22eaaf374f80d0d2ff28d1c.jpg  
 extracting: ../data/train/950_jpg.rf.a243613cc6e72221ab68c9a5670d07d1.jpg  
 extracting: ../data/train/951_jpg.rf.16a1458b68d03eaf8d0cbd7c03836d13.jpg  
 extracting: ../data/train/952_jpg.rf.19cfa3409a4041bf06c069fb087a4368.jpg  
 extracting: ../data/train/954_jpg.rf.2e8c00ff4c1fadf6df19e92feb700694.jpg  
 extracting: ../data/train/955_jpg.rf.6964c8fbf5e8e5631204316a5dec8860.jpg  
 extracting: ../data/train/956_jpg.rf.2999012c89b47efd524e5fe1ec861877.jpg  
 extracting: ../data/train/958_jpg.rf.4a4e1b37edbbd9c78be2b7dc351a013d.jpg  
 extracting: ../data/train/959_jpg.rf.2e6d7c36a3c329fc05f535f3a2568515.jpg  


 extracting: ../data/valid/1914_jpg.rf.295b7925c71b727f4d7b6dbde8443560.jpg  
 extracting: ../data/valid/1918_jpg.rf.f044708ab5bd41011ca1f6706debf6e7.jpg  
 extracting: ../data/valid/1920_jpg.rf.8b3ebdc4092ed97be06458add38fefd4.jpg  
 extracting: ../data/valid/1925_jpg.rf.c53d7af54308e8cd37d49410082af219.jpg  
 extracting: ../data/valid/1937_jpg.rf.650b0164b6dc6a21aca8a402889a7605.jpg  
 extracting: ../data/valid/1939_jpg.rf.bac4ed677a3b19f661da1db2504b2f64.jpg  
 extracting: ../data/valid/1943_jpg.rf.fa9fb139707f982f1845578a11fab553.jpg  
 extracting: ../data/valid/1947_jpg.rf.c9558744e59fb0670f97e184accf0dba.jpg  
 extracting: ../data/valid/1948_jpg.rf.91d3a586bbfdb3c9706315f5f4de19d8.jpg  
 extracting: ../data/valid/1960_jpg.rf.633d3cfcc6fef79ae67614306ab1ea0d.jpg  
 extracting: ../data/valid/1962_jpg.rf.a40f767462f8d083b8d09438438b8c67.jpg  
 extracting: ../data/valid/1971_jpg.rf.e616b708394477b5811504a85a61409d.jpg  
 extracting: ../data/valid/1978_jpg.rf.8c318111a5f85065b8ca1eb8c

 extracting: ../data/valid/3115_jpg.rf.0861760fd983a2fa4d1619718cde1920.jpg  
 extracting: ../data/valid/3116_jpg.rf.b00700febe761f12c6c001f994db8ee8.jpg  
 extracting: ../data/valid/311_jpg.rf.cc2b5db6a1388788261c2e8e93d1b9c7.jpg  
 extracting: ../data/valid/3131_jpg.rf.54f2dda9236698cdc06a761f1e55c25f.jpg  
 extracting: ../data/valid/3143_jpg.rf.e2e80f7b48361e68050a2443435fe64c.jpg  
 extracting: ../data/valid/3150_jpg.rf.c90457f69dba8ca92fde63dfe6a3efa3.jpg  
 extracting: ../data/valid/3152_jpg.rf.30df3eff76c2507da12bd052c4038464.jpg  
 extracting: ../data/valid/3155_jpg.rf.e899b4c87f28c224bd747849f0521c96.jpg  
 extracting: ../data/valid/3163_jpg.rf.5d26f0a9345f619fbdd289f60a9d1d2c.jpg  
 extracting: ../data/valid/3168_jpg.rf.5146087c251d2ddc2275b078cb300aa5.jpg  
 extracting: ../data/valid/3169_jpg.rf.39ffd9277db2cb8a35c5bace9501b3d8.jpg  
 extracting: ../data/valid/3172_jpg.rf.88c29723f71cf13e0a987e0b3d4c5e23.jpg  
 extracting: ../data/valid/3174_jpg.rf.46431a9bdabed0b70dbed7b438

 extracting: ../data/valid/4233_jpg.rf.9199b3eaa11c77c78b47f4099b5bf3ae.jpg  
 extracting: ../data/valid/4244_jpg.rf.bb02013f449ca22ac5b80ce11696f492.jpg  
 extracting: ../data/valid/4261_jpg.rf.813244f149f34368388b02e251d10e3c.jpg  
 extracting: ../data/valid/4266_jpg.rf.3c648c4d37dc566bffd568b19cf9df45.jpg  
 extracting: ../data/valid/4267_jpg.rf.64b6229d9258268244029ec0c76d08a6.jpg  
 extracting: ../data/valid/4269_jpg.rf.1ca0a4644ec4374f114c0e0cdd85cb10.jpg  
 extracting: ../data/valid/4270_jpg.rf.b4a68be6691e57a2170408e252660cfe.jpg  
 extracting: ../data/valid/4273_jpg.rf.34e948a234558a765be27f2fd56e7439.jpg  
 extracting: ../data/valid/4277_jpg.rf.60aa02473e24876e0666474f0ee4908c.jpg  
 extracting: ../data/valid/4278_jpg.rf.22bb0d863099a34ea9eb669d880bb30c.jpg  
 extracting: ../data/valid/4282_jpg.rf.257dfb997246c00c65471304961722e0.jpg  
 extracting: ../data/valid/4283_jpg.rf.e668e4d1e051365dff6e01dc628f9a52.jpg  
 extracting: ../data/valid/4285_jpg.rf.88db18096139633ce09fdef8b

## Model

In [3]:
retinanet_repo = 'https://github.com/fizyr/keras-retinanet.git'
retinanet_dir = 'keras-retinanet'

if not exists(retinanet_dir):
    !git clone -q --recurse-submodules --depth 1 $retinanet_repo
    !cd {retinanet_dir} && pip install . --user

In [21]:
import csv
class_list = []
        
with open('../data/train/_annotations.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        fruit = row[-1]
        if fruit not in class_list:
            class_list.append(fruit)
            
class_list = sorted(class_list)

with open('../data/classes.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for i in range(len(class_list)):
        writer.writerow([class_list[i]] + [str(i)])

In [ ]:
!retinanet-train csv '../data/train/_annotations.csv' '../data/classes.csv'

Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/ai/bin/retinanet-train", line 8, in <module>
    sys.exit(main())
  File "/home/raul/.local/lib/python3.7/site-packages/keras_retinanet/bin/train.py", line 483, in main
    train_generator, validation_generator = create_generators(args, backbone.preprocess_image)
  File "/home/raul/.local/lib/python3.7/site-packages/keras_retinanet/bin/train.py", line 303, in create_generators
    **common_args
  File "/home/raul/.local/lib/python3.7/site-packages/keras_retinanet/preprocessing/csv_generator.py", line 161, in __init__
    super(CSVGenerator, self).__init__(**kwargs)
  File "/home/raul/.local/lib/python3.7/site-packages/keras_retinanet/preprocessing/generator.py", line 89, in __init__
    self.group_images()
  File "/home/raul/.local/lib/python3.7/site-packages/keras_retinanet/preprocessing/generator.py", line 291, in group_images
    order.sort(key=lambda x: self.image_aspect_ratio(x))
  File "/home/raul/.local/lib/py